High Performance Python
=======

http://tiny.cc/dm3wgz




Lecture 7. Numba
----------

Heavily based on (forked): 

Scipy2017 tutorial by Gil Forsyth:

https://github.com/gforsyth/numba_tutorial_scipy2017

https://www.youtube.com/watch?v=1AwG0T4gaO0&t=1349s

GTC2018 tutorial by Stan Seibert:

https://github.com/ContinuumIO/gtc2018-numba


High Performance Python
----------------------

* multiprocessing
* mpi4py
* pycuda
* cupy
* pyopencl
* **numba**

Numba is:

Just-In-Time (JIT) compiler:
* generates optimized machine code using LLVM
* integrates well with Scientific Python stack
* **function compiler**: Numba compiles Python functions (not entire applications and not parts of functions). Numba is a Python module.
* **type-specializing**: Numba speeds up your function by generating a specialized implementation for the specific data types you are using. 
* **just-in-time**: Numba translates functions when they are first called so that the compiler knows the argument types. Works in Jupyter notebook.
* **numerically-focused**: „mostly“ int, float, complex. Works good with numpy arrays.


The first step is always to find the bottlenecks in your code, via _profiling_: analyzing your code by measuring the execution time of its parts.


Tools:
------

2. `cProfile`
4. `snakeviz`
1. [`line_profiler`](https://github.com/rkern/line_profiler)
3. `timeit`



```console
pip install line_profiler
```

In [1]:
import numpy
from time import sleep

def sleepy(time2sleep):
    sleep(time2sleep)
    
def supersleepy(time2sleep):
    sleep(time2sleep)
    
def randmatmul(n=1000):
    a = numpy.random.random((n,n))
    b = a @ a
    return b
    
def useless(a):
    if not isinstance(a, int):
        return
    
    randmatmul(a)
    
    ans = 0
    for i in range(a):
        ans += i
        
    sleepy(1.0)
    supersleepy(2.0)
        
    return ans

## using `cProfile`

[`cProfile`](https://docs.python.org/3.4/library/profile.html#module-cProfile) is the built-in profiler in Python (available since Python 2.5).  It provides a function-by-function report of execution time. First import the module, then usage is simply a call to `cProfile.run()` with your code as argument. It will print out a list of all the functions that were called, with the number of calls and the time spent in each.


In [0]:
import cProfile

cProfile.run('useless(3000)')

         11 function calls in 4.739 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.621    1.621    1.735    1.735 <ipython-input-1-3d4635a3d65c>:10(randmatmul)
        1    0.001    0.001    4.739    4.739 <ipython-input-1-3d4635a3d65c>:15(useless)
        1    0.000    0.000    1.001    1.001 <ipython-input-1-3d4635a3d65c>:4(sleepy)
        1    0.000    0.000    2.002    2.002 <ipython-input-1-3d4635a3d65c>:7(supersleepy)
        1    0.000    0.000    4.739    4.739 <string>:1(<module>)
        1    0.000    0.000    4.739    4.739 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        2    3.003    1.502    3.003    1.502 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.114    0.114    0.114    0.114 {method 'random' of 'numpy.random.mtrand.RandomS

## using `snakeviz`

In [2]:
%load_ext snakeviz

ModuleNotFoundError: No module named 'snakeviz'

In [0]:
%snakeviz useless(3000)

## using `line_profiler`

`line_profiler` offers more granular information than `cProfile`: it will give timing information about each line of code in a profiled function.

### For a pop-up window with results in notebook:

IPython has an `%lprun` magic to profile specific functions within an executed statement. Usage:
`%lprun -f func_to_profile <statement>` (get more help by running `%lprun?` in IPython).

In [0]:
%load_ext line_profiler
%lprun -f sleepy -f supersleepy useless(1000)

ModuleNotFoundError: ignored

### Write results to a text file

In [3]:
%lprun -T timings.txt -f sleepy useless(1000)

UsageError: Line magic function `%lprun` not found.


## Profiling on the command line

Open file, add `@profile` decorator to any function you want to profile, then run

```console
kernprof -l script_to_profile.py
```

which will generate `script_to_profile.py.lprof` (pickled result).  To view the results, run

```console
python -m line_profiler script_to_profile.py.lprof
```

In [0]:
from IPython.display import IFrame
IFrame('http://localhost:8888/terminals/1', width=800, height=700)

## `timeit`

```python
python -m timeit "print(42)"
```


In [4]:
# line magic
%timeit x=10

7.58 ns ± 0.243 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [5]:
%%timeit 
# cell magic

x=10
a='hello'
d=[1,2,3]

38.1 ns ± 2.24 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


JIT
===

### Array sum

The function below is a naive `sum` function that sums all the elements of a given array.

In [0]:
def sum_array(inp):
    J, I = inp.shape
    
    #this is a bad idea
    mysum = 0
    for j in range(J):
        for i in range(I):
            mysum += inp[j, i]
            
    return mysum

In [0]:
import numpy
arr = numpy.random.random((300, 300))

sum_array(arr)

plain = %timeit -o sum_array(arr)

10 loops, best of 3: 23.8 ms per loop


In [0]:
from numba import jit

sum_array_numba = jit()(sum_array)

sum_array_numba(arr)

jitted = %timeit -o sum_array_numba(arr)

plain.best / jitted.best


10000 loops, best of 3: 137 µs per loop


173.82114138359057

## More commonly as a decorator

In [0]:
@jit
def sum_array(inp):
    I, J = inp.shape
    
    mysum = 0
    for i in range(I):
        for j in range(J):
            mysum += inp[i, j]
            
    return mysum

In [0]:
%timeit arr.sum()

The slowest run took 6.72 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 32.7 µs per loop


Exercise 1. JIT the Mandelbrot fractal
--------------------------------------------

Separate exercise notebook or use your own Mandelbrot code 

1. Profile the code, find the bottlenecks
2. Use Numba to speed up the code
3. Compare the timing

In [6]:
@jit
def add(a, b):
    return a + b

NameError: name 'jit' is not defined

In [0]:
add(1, 1)

2

Numba examines Python bytecode and then translates this into an 'intermediate representation'.  To view this IR, run (compile) `add` and you can access the `inspect_types` method.

In [0]:
add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-10-7a2ac56f16b6>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $0.3 = a + b  :: int64
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return a + b




Ok.  Numba is has correctly inferred the type of the arguments, defining things as `int64` and running smoothly.  

(What happens if you do `add(1., 1.)` and then `inspect_types`?)

In [0]:
add(1.,1.)

2.0

In [0]:
add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-10-7a2ac56f16b6>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $0.3 = a + b  :: int64
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return a + b


add (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-10-7a2ac56f16b6>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: float64
    #   b = arg(1, name=b)  :: float64
    #   $0.3 = a + b  :: float64
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: float64
    #   del $0.3
    #   return $0.4

    return a + b




### What about the actual LLVM code?

You can see the actual LLVM code generated by Numba using the `inspect_llvm()` method.  Since it's a `dict`, doing the following will be slightly more visually friendly.

In [0]:
for k, v in add.inspect_llvm().items():
    print(k, v)

(int64, int64) ; ModuleID = 'add'
source_filename = "<string>"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@"_ZN08NumbaEnv8__main__7add$242Exx" = common local_unnamed_addr global i8* null
@.const.add = internal constant [4 x i8] c"add\00"
@PyExc_RuntimeError = external global i8
@".const.missing Environment" = internal constant [20 x i8] c"missing Environment\00"

; Function Attrs: norecurse nounwind writeonly
define i32 @"_ZN8__main__7add$242Exx"(i64* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i64 %arg.a, i64 %arg.b) local_unnamed_addr #0 {
entry:
  %.14 = add nsw i64 %arg.b, %arg.a
  store i64 %.14, i64* %retptr, align 8
  ret i32 0
}

define i8* @"_ZN7cpython8__main__7add$242Exx"(i8* nocapture readnone %py_closure, i8* %py_args, i8* nocapture readnone %py_kws) local_unnamed_addr {
entry:
  %.5 = alloca i8*, align 8
  %.6 = alloca i8*, align 8
  %.7 = call i32 (i8*, i8*, i64, i64, ...) @Py

## But there's a caveat
Now, watch what happens when we try to do something that is natural in Python, but not particularly mathematically sound:

In [0]:
from numba import njit

In [0]:
def add_strings(a, b):
    return a + b

In [0]:
add_strings_jit = njit()(add_strings)

In [0]:
add_strings_jit('a', 'b')

TypingError: ignored

In [0]:
add_strings_jit.inspect_types()

### Compiler options

```python
cache=True
```

if you don't want to always want to get dinged by the compilation time for every run. This will actually save the compiled function into something like a `pyc` file in your `__pycache__` directory, so even between sessions you should have nice fast performance.

## N-Body problems

Many physical problems require the evaluation of all pairwise interactions of a large number of particles, so-called N-body problems. These problems arise in molecular dynamics, astrodynamics and electromagnetics among others. 

Their pairwise interactions can be expressed as:

\begin{equation}
f_i = \sum_{j=1}^n{P \left(\boldsymbol{x}_i, \boldsymbol{x}_j \right)w_j} \ \ \ \text{for } i=1,2,...,n 
\end{equation}

*  where subscripts $i$,  $j$ respectively denote *target* and *source*
*  $f_i$ can be a *potential* (or *force*) at target point $i$
*  $w_j$ is the *source weight* 
*  $\boldsymbol{x}_i, \boldsymbol{x}_j$ are the *spatial positions* of particles 
*  $P \left(\boldsymbol{x}_i, \boldsymbol{x}_j \right)$ is the *interaction kernel*. 

In order to evalute the potential $f_i$ at a target point $i$, we have to loop over each source particle $j$. Since there are $n$ target points $i$, this 'brute-force' approach costs $\mathcal{O} \left(n^2 \right)$ operations. 

One possible approach in this kind of problem is to define a few classes, say `Point` and `Particle` and then loop over the objects and perform the necessary point-to-point calculations.

In [0]:
class Point():
    """    
    Arguments:
        domain: the domain of random generated coordinates x,y,z, 
                default=1.0
    
    Attributes:
        x, y, z: coordinates of the point
    """
    def __init__(self, domain=1.0):
        self.x = domain * numpy.random.random()
        self.y = domain * numpy.random.random()
        self.z = domain * numpy.random.random()
            
    def distance(self, other):
        return ((self.x - other.x)**2 + 
                (self.y - other.y)**2 + 
                (self.z - other.z)**2)**.5

In [0]:
class Particle(Point):
    """    
    Attributes:
        m: mass of the particle
        phi: the potential of the particle
    """
    
    def __init__(self, domain=1.0, m=1.0):
        Point.__init__(self, domain)
        self.m = m
        self.phi = 0.

Now we create a list of `n` random particles, define a function to calculate their interaction via direct summation and run!

In [0]:
n = 1000
particles = [Particle(m = 1 / n) for i in range(n)]

In [0]:
def direct_sum(particles):
    """
    Calculate the potential at each particle
    using direct summation method.

    Arguments:
        particles: the list of particles

    """
    for i, target in enumerate(particles):
        for source in (particles[:i] + particles[i+1:]):
            r = target.distance(source)
            target.phi += source.m / r

In [0]:
direct_sum(particles)

In [0]:
orig_time = %timeit -o direct_sum(particles)

In [0]:
%load_ext line_profiler

In [0]:
%lprun -f direct_sum direct_sum(particles)

## How do we use Numba on this problem?
Problem: Numba doesn't support jitting native Python classes.  There is a `jit_class` structure in Numba but it's still in early development.

But it's nice to have attributes for literate programming.

Solution: NumPy custom dtypes.

In [0]:
particle_dtype = numpy.dtype({'names':['x','y','z','m','phi'], 
                             'formats':[numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double, 
                                        numpy.double]})

In [0]:
myarray = numpy.ones(3, dtype=particle_dtype)

In [0]:
myarray

In [0]:
myarray[0]['x'] = 2.0

In [0]:
myarray

### Exercise 1

Write a function `create_n_random_particles` that takes the arguments `n` (number of particles), `m` (mass of every particle) and a domain within to generate a random number (as in the class above).
It should create an array with `n` elements and `dtype=particle_dtype` and then return that array.

For each particle, the mass should be initialized to the value of `m` and the potential `phi` initialized to zero.

For the `x` component of a given particle `p`, you might do something like

```python
p['x'] = domain * numpy.random.random()
```

In [0]:
@njit
def create_n_random_particles(n, m, domain=1):
    '''
    Creates `n` particles with mass `m` with random coordinates
    between 0 and `domain`
    '''
    parts = numpy.zeros((n), dtype=particle_dtype)


    ### your code
    ### your code
    ### your code
    
    return parts   #parts is an array of particles

In [0]:
# test it
parts = create_n_random_particles(1000, .001, 1)

### Exercise 2

Write a JITted function `distance` to calculate the distance between two particles of dtype `particle_dtype`

Here's the `distance` method from the `Particle` class as a reference:

```python
def distance(self, other):
        return ((self.x - other.x)**2 + 
                (self.y - other.y)**2 + 
                (self.z - other.z)**2)**.5
```

In [0]:
def distance(part1, part2):
    '''calculate the distance between two particles'''
    
    # your code here

In [0]:
# test it

distance(parts[0], parts[1])

### Exercise 3
Modify the original `direct_sum` function (copied below for reference) to instead work a NumPy array of particles.  Loop over each element in the array and calculate its total potential.

```python
def direct_sum(particles):
    """
    Calculate the potential at each particle
    using direct summation method.

    Arguments:
        particles: the list of particles

    """
    for i, target in enumerate(particles):
        for source in (particles[:i] + particles[i+1:]):
            r = target.distance(source)
            target.phi += source.m / r

In [0]:
def direct_sum(particles):
    # take it away